In [7]:
!mkdir -p homework && cd homework
!pip install uv

!curl -LsSf https://astral.sh/uv/install.sh | sh
#Q1
!uv --version

downloading uv 0.9.7 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
uv 0.9.7


In [9]:
!uv init .

Initialized project `content` at `/content`


In [22]:
#Q2
!pip install uv -q

!uv add scikit-learn==1.6.1
!python - <<'PY'
import re, pathlib
t = pathlib.Path("uv.lock").read_text()
m = re.search(r"sha256:[0-9a-f]+", t)
print(m.group(0) if m else "NOT_FOUND")
!PY

Resolved 6 packages in 0.91ms
Audited 5 packages in 0.23ms
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
sha256:3faa5c39054b2f03ca547da9b2f52fde67c06240c31853f306aea97f13647b55
/bin/bash: line 1: PY: command not found


In [24]:
!wget -q https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

In [26]:
#Q3
!uv run python - << 'PY'
import pickle, json
with open("pipeline_v1.bin","rb") as f:
    model=pickle.load(f)
x={"lead_source":"paid_ads","number_of_courses_viewed":2,"annual_income":79276.0}
p=model.predict_proba([x])[0,1]
print(round(float(p),3))
!PY

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
0.534
/bin/bash: line 1: PY: command not found


In [ ]:
!pip -q install fastapi uvicorn requests pydantic
[ -f pipeline_v1.bin ] || wget -q https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

In [33]:
open("app.py","w").write(
"import pickle\n"
"from fastapi import FastAPI\n"
"from pydantic import BaseModel\n"
"with open('pipeline_v1.bin','rb') as f:\n"
"    model=pickle.load(f)\n"
"class Lead(BaseModel):\n"
"    lead_source: str\n"
"    number_of_courses_viewed: int\n"
"    annual_income: float\n"
"app=FastAPI()\n"
"@app.post('/predict')\n"
"def predict(lead: Lead):\n"
"    p=model.predict_proba([lead.model_dump()])[0,1]\n"
"    return {'probability': float(p)}\n"
)

391

In [36]:
#Q4
!nohup python -m uvicorn app:app --host 127.0.0.1 --port 8000 >/dev/null 2>&1 &
!sleep 2
!python - <<'PY'
import requests, json, time
for _ in range(10):
    try:
        r=requests.post("http://127.0.0.1:8000/predict",json={"lead_source":"organic_search","number_of_courses_viewed":4,"annual_income":80304.0},timeout=2)
        print(r.json())
        break
    except Exception as e:
        time.sleep(0.5)
!PY

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
{'probability': 0.5340417283801275}
/bin/bash: line 1: PY: command not found


In [ ]:
#Q5
!docker pull agrigorev/zoomcamp-model:2025
!docker images | grep zoomcamp-model

In [ ]:
#Q6
import pickle
from fastapi import FastAPI
from pydantic import BaseModel
with open("pipeline_v2.bin","rb") as f:
    model=pickle.load(f)
class Lead(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float
app=FastAPI()
@app.post("/predict")
def predict(lead: Lead):
    p=model.predict_proba([lead.model_dump()])[0,1]
    return {"probability": float(p)}

In [ ]:
[project]
name = "hw5-app"
version = "0.1.0"
dependencies = [
  "fastapi",
  "uvicorn",
  "pydantic",
  "scikit-learn==1.6.1",
]

In [ ]:
FROM agrigorev/zoomcamp-model:2025
WORKDIR /code
COPY pyproject.toml uv.lock* ./
RUN pip install --no-cache-dir uv && uv sync || (pip install --no-cache-dir fastapi uvicorn pydantic scikit-learn==1.6.1)
COPY app_docker.py ./
EXPOSE 8000
CMD ["python","-m","uvicorn","app_docker:app","--host","0.0.0.0","--port","8000"]

In [ ]:
docker build -t hw5-app .
docker run --rm -p 8000:8000 hw5-app

In [39]:
#for test
!python - <<'PY'
import requests
u="http://127.0.0.1:8000/predict"
c={"lead_source":"organic_search","number_of_courses_viewed":4,"annual_income":80304.0}
print(requests.post(u,json=c,timeout=5).json())
!PY

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
{'probability': 0.5340417283801275}
/bin/bash: line 1: PY: command not found
